In [2]:
import gzip # pour décompresser les données
import pickle 
import matplotlib.pyplot as plt # pour l'affichage
import torch,torch.utils.data
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

In [7]:
# Recuperation des données

# nombre d'image lues à chaque fois dans la base d'apprentissage (laisser à 1 sauf pour la question optionnelle sur les minibatchs)
TRAIN_BATCH_SIZE = 1
# on charge les données de la base MNIST
data = pickle.load(gzip.open('mnist.pkl.gz'),encoding='latin1')
# images de la base d'apprentissage
train_data = data[0][0]
# labels de la base d'apprentissage
train_data_label = data[0][1]
# images de la base de test
test_data = data[1][0]
# labels de la base de test
test_data_label = data[1][1]

In [3]:
# Partie 1 : Perceptron

weight = 0.5
nb_step = 0.001
w = np.full((785, 10), weight)

for d in range(len(train_data)):
    x = train_data[d].numpy()
    x = np.append(x, 1)
    y = np.dot(x, w)
    dl = train_data_label[d].numpy()
    a = nb_step * (dl - y)
    b = np.outer(x, a)
    w += b

nbCorrect = 0
for d in range(len(test_data)):
    x = test_data[d].numpy()
    x = np.append(x, 1)
    y = np.dot(x, w)
    index = np.argmax(test_data_label[d])
    index2 = np.argmax(y)
    if index == index2:
        nbCorrect += 1
print(nbCorrect/len(test_data) * 100,"%")

83.71428571428572 %


In [8]:
# Partie 2 : Shallow network
nb_hn = 150
nb_step = 0.001
w_h = np.zeros((np.shape(train_data)[1] + 1, nb_hn))
w_o = np.random.randn(nb_hn, np.shape(train_data_label)[1])
for i in range(5):
    for d in range(len(train_data)):
        # Choisir entrée
        x = train_data[d]
        x = np.append(x, 1.)

        # Propagation yi(1)
        y_h = 1. / (1. + np.exp(-np.dot(x, w_h)))

        # Propagation yi(2)
        y_o = np.dot(y_h, w_o)

        # Rétro Propagation 

        # Calcul de l'erreur
        dl = train_data_label[d]
        m_o = dl - y_o
        
        # Rétro Propa de l'erreur
        a1 = y_h * (1 - y_h)
        a2 = np.dot(w_o, m_o)
        m_h = a1 * a2
        
        # Modification poids
        dw_o = nb_step * np.outer(y_h, m_o)
        w_o += dw_o
        dw_h = nb_step * np.outer(x, m_h)
        w_h += dw_h

In [9]:
nbCorrect = 0
for d in range(len(test_data)):
    x = test_data[d]
    x = np.append(x, 1)

    y_h = 1. / (1. + np.exp(-np.dot(x, w_h)))

    y_o = np.dot(y_h, w_o)
    index = np.argmax(test_data_label[d])
    index2 = np.argmax(y_o)
    if index == index2:
        nbCorrect += 1
print(nbCorrect/len(test_data) * 100,"%")

94.34285714285714 %


In [13]:
#  Partie 3 : Deep network

# Import as Tensors

TRAIN_BATCH_SIZE = 64

data = pickle.load(gzip.open('mnist.pkl.gz'),encoding='latin1')
train_data = torch.Tensor(data[0][0])
train_data_label = torch.Tensor(data[0][1])
test_data = torch.Tensor(data[1][0])
test_data_label = torch.Tensor(data[1][1])
train_dataset = torch.utils.data.TensorDataset(train_data,train_data_label)
test_dataset = torch.utils.data.TensorDataset(test_data,test_data_label)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

In [93]:
class DeepNN(nn.Module):
    def __init__(self, layers_size, activation_function):
        super(DeepNN, self).__init__()
        self.activation_function = activation_function
        self.layers = nn.ModuleList()
        #create the layers
        for i in range(len(layers_size) - 1):
            layer = nn.Linear(layers_size[i], layers_size[i+1])
            self.layers.append(layer)

    def forward(self, x):
        for l in self.layers:
            x = self.activation_function(l(x))
        return x
            
nb_step = 0.001
epochs = 6
layers_size = [784, 150 ,30, 10]
activation_function = nn.Sigmoid()
#activation_function = nn.ReLU()
log = True

deepNN = DeepNN(layers_size, activation_function)
optimizer = torch.optim.Adam(deepNN.parameters(), lr=nb_step)

for epoch in range(epochs):
    deepNN.train()
    for batch_ep, (image,label) in enumerate(train_loader):
        optimizer.zero_grad()
        output = deepNN(image)
        loss_f = nn.MSELoss()
        loss = loss_f(output, label)
        loss.backward()
        optimizer.step()
        if log:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_ep * len(data), len(train_loader.dataset),
                100. * batch_ep / len(train_loader), loss.item()))
            
deepNN.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for image, label in test_loader:
        output = deepNN(image)
        lose_f = nn.MSELoss()
        test_loss += lose_f(output, label).item() 
        pred = output.argmax(dim=1, keepdim=True)
        correct += int(np.argmax(output)) == int(np.argmax(label))

test_loss /= len(test_loader.dataset)

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

Train Epoch: 0 [0/63000 (0%)]	Loss: 0.263570
Train Epoch: 0 [2/63000 (0%)]	Loss: 0.258656
Train Epoch: 0 [4/63000 (0%)]	Loss: 0.250883
Train Epoch: 0 [6/63000 (0%)]	Loss: 0.242127
Train Epoch: 0 [8/63000 (0%)]	Loss: 0.235807
Train Epoch: 0 [10/63000 (1%)]	Loss: 0.229645
Train Epoch: 0 [12/63000 (1%)]	Loss: 0.222335
Train Epoch: 0 [14/63000 (1%)]	Loss: 0.219818
Train Epoch: 0 [16/63000 (1%)]	Loss: 0.209957
Train Epoch: 0 [18/63000 (1%)]	Loss: 0.208473
Train Epoch: 0 [20/63000 (1%)]	Loss: 0.202438
Train Epoch: 0 [22/63000 (1%)]	Loss: 0.194875
Train Epoch: 0 [24/63000 (1%)]	Loss: 0.192975
Train Epoch: 0 [26/63000 (1%)]	Loss: 0.188871
Train Epoch: 0 [28/63000 (1%)]	Loss: 0.184866
Train Epoch: 0 [30/63000 (2%)]	Loss: 0.181863
Train Epoch: 0 [32/63000 (2%)]	Loss: 0.176568
Train Epoch: 0 [34/63000 (2%)]	Loss: 0.172499
Train Epoch: 0 [36/63000 (2%)]	Loss: 0.169788
Train Epoch: 0 [38/63000 (2%)]	Loss: 0.165868
Train Epoch: 0 [40/63000 (2%)]	Loss: 0.162980
Train Epoch: 0 [42/63000 (2%)]	Loss: 0.

Train Epoch: 0 [686/63000 (35%)]	Loss: 0.087783
Train Epoch: 0 [688/63000 (35%)]	Loss: 0.088695
Train Epoch: 0 [690/63000 (35%)]	Loss: 0.089289
Train Epoch: 0 [692/63000 (35%)]	Loss: 0.089207
Train Epoch: 0 [694/63000 (35%)]	Loss: 0.088923
Train Epoch: 0 [696/63000 (35%)]	Loss: 0.088710
Train Epoch: 0 [698/63000 (35%)]	Loss: 0.088305
Train Epoch: 0 [700/63000 (36%)]	Loss: 0.087366
Train Epoch: 0 [702/63000 (36%)]	Loss: 0.088268
Train Epoch: 0 [704/63000 (36%)]	Loss: 0.087634
Train Epoch: 0 [706/63000 (36%)]	Loss: 0.087936
Train Epoch: 0 [708/63000 (36%)]	Loss: 0.088259
Train Epoch: 0 [710/63000 (36%)]	Loss: 0.089202
Train Epoch: 0 [712/63000 (36%)]	Loss: 0.089084
Train Epoch: 0 [714/63000 (36%)]	Loss: 0.087648
Train Epoch: 0 [716/63000 (36%)]	Loss: 0.088610
Train Epoch: 0 [718/63000 (36%)]	Loss: 0.087804
Train Epoch: 0 [720/63000 (37%)]	Loss: 0.088584
Train Epoch: 0 [722/63000 (37%)]	Loss: 0.088247
Train Epoch: 0 [724/63000 (37%)]	Loss: 0.088895
Train Epoch: 0 [726/63000 (37%)]	Loss: 0

Train Epoch: 0 [1360/63000 (69%)]	Loss: 0.080812
Train Epoch: 0 [1362/63000 (69%)]	Loss: 0.078459
Train Epoch: 0 [1364/63000 (69%)]	Loss: 0.080684
Train Epoch: 0 [1366/63000 (69%)]	Loss: 0.085926
Train Epoch: 0 [1368/63000 (69%)]	Loss: 0.082069
Train Epoch: 0 [1370/63000 (70%)]	Loss: 0.080083
Train Epoch: 0 [1372/63000 (70%)]	Loss: 0.081470
Train Epoch: 0 [1374/63000 (70%)]	Loss: 0.081570
Train Epoch: 0 [1376/63000 (70%)]	Loss: 0.083135
Train Epoch: 0 [1378/63000 (70%)]	Loss: 0.076717
Train Epoch: 0 [1380/63000 (70%)]	Loss: 0.081015
Train Epoch: 0 [1382/63000 (70%)]	Loss: 0.080816
Train Epoch: 0 [1384/63000 (70%)]	Loss: 0.082335
Train Epoch: 0 [1386/63000 (70%)]	Loss: 0.078994
Train Epoch: 0 [1388/63000 (70%)]	Loss: 0.080983
Train Epoch: 0 [1390/63000 (71%)]	Loss: 0.079472
Train Epoch: 0 [1392/63000 (71%)]	Loss: 0.083815
Train Epoch: 0 [1394/63000 (71%)]	Loss: 0.077452
Train Epoch: 0 [1396/63000 (71%)]	Loss: 0.080438
Train Epoch: 0 [1398/63000 (71%)]	Loss: 0.079010
Train Epoch: 0 [1400

Train Epoch: 1 [62/63000 (3%)]	Loss: 0.060800
Train Epoch: 1 [64/63000 (3%)]	Loss: 0.060742
Train Epoch: 1 [66/63000 (3%)]	Loss: 0.056318
Train Epoch: 1 [68/63000 (3%)]	Loss: 0.057198
Train Epoch: 1 [70/63000 (4%)]	Loss: 0.053986
Train Epoch: 1 [72/63000 (4%)]	Loss: 0.057944
Train Epoch: 1 [74/63000 (4%)]	Loss: 0.065866
Train Epoch: 1 [76/63000 (4%)]	Loss: 0.059648
Train Epoch: 1 [78/63000 (4%)]	Loss: 0.053105
Train Epoch: 1 [80/63000 (4%)]	Loss: 0.057623
Train Epoch: 1 [82/63000 (4%)]	Loss: 0.060967
Train Epoch: 1 [84/63000 (4%)]	Loss: 0.052190
Train Epoch: 1 [86/63000 (4%)]	Loss: 0.056657
Train Epoch: 1 [88/63000 (4%)]	Loss: 0.058533
Train Epoch: 1 [90/63000 (5%)]	Loss: 0.054742
Train Epoch: 1 [92/63000 (5%)]	Loss: 0.058473
Train Epoch: 1 [94/63000 (5%)]	Loss: 0.057202
Train Epoch: 1 [96/63000 (5%)]	Loss: 0.048251
Train Epoch: 1 [98/63000 (5%)]	Loss: 0.053876
Train Epoch: 1 [100/63000 (5%)]	Loss: 0.054746
Train Epoch: 1 [102/63000 (5%)]	Loss: 0.047314
Train Epoch: 1 [104/63000 (5%)]	

Train Epoch: 1 [746/63000 (38%)]	Loss: 0.028275
Train Epoch: 1 [748/63000 (38%)]	Loss: 0.032272
Train Epoch: 1 [750/63000 (38%)]	Loss: 0.034476
Train Epoch: 1 [752/63000 (38%)]	Loss: 0.040924
Train Epoch: 1 [754/63000 (38%)]	Loss: 0.037255
Train Epoch: 1 [756/63000 (38%)]	Loss: 0.029736
Train Epoch: 1 [758/63000 (38%)]	Loss: 0.030960
Train Epoch: 1 [760/63000 (39%)]	Loss: 0.039161
Train Epoch: 1 [762/63000 (39%)]	Loss: 0.029057
Train Epoch: 1 [764/63000 (39%)]	Loss: 0.031069
Train Epoch: 1 [766/63000 (39%)]	Loss: 0.037157
Train Epoch: 1 [768/63000 (39%)]	Loss: 0.029212
Train Epoch: 1 [770/63000 (39%)]	Loss: 0.029720
Train Epoch: 1 [772/63000 (39%)]	Loss: 0.035280
Train Epoch: 1 [774/63000 (39%)]	Loss: 0.033505
Train Epoch: 1 [776/63000 (39%)]	Loss: 0.028305
Train Epoch: 1 [778/63000 (39%)]	Loss: 0.038117
Train Epoch: 1 [780/63000 (40%)]	Loss: 0.038503
Train Epoch: 1 [782/63000 (40%)]	Loss: 0.034467
Train Epoch: 1 [784/63000 (40%)]	Loss: 0.035748
Train Epoch: 1 [786/63000 (40%)]	Loss: 0

...

Train Epoch: 4 [922/63000 (47%)]	Loss: 0.005812
Train Epoch: 4 [924/63000 (47%)]	Loss: 0.013358
Train Epoch: 4 [926/63000 (47%)]	Loss: 0.007743
Train Epoch: 4 [928/63000 (47%)]	Loss: 0.007528
Train Epoch: 4 [930/63000 (47%)]	Loss: 0.007348
Train Epoch: 4 [932/63000 (47%)]	Loss: 0.008044
Train Epoch: 4 [934/63000 (47%)]	Loss: 0.007670
Train Epoch: 4 [936/63000 (48%)]	Loss: 0.002808
Train Epoch: 4 [938/63000 (48%)]	Loss: 0.009352
Train Epoch: 4 [940/63000 (48%)]	Loss: 0.007678
Train Epoch: 4 [942/63000 (48%)]	Loss: 0.003038
Train Epoch: 4 [944/63000 (48%)]	Loss: 0.007747
Train Epoch: 4 [946/63000 (48%)]	Loss: 0.012327
Train Epoch: 4 [948/63000 (48%)]	Loss: 0.004342
Train Epoch: 4 [950/63000 (48%)]	Loss: 0.004851
Train Epoch: 4 [952/63000 (48%)]	Loss: 0.006549
Train Epoch: 4 [954/63000 (48%)]	Loss: 0.011845
Train Epoch: 4 [956/63000 (49%)]	Loss: 0.008695
Train Epoch: 4 [958/63000 (49%)]	Loss: 0.008949
Train Epoch: 4 [960/63000 (49%)]	Loss: 0.002816
Train Epoch: 4 [962/63000 (49%)]	Loss: 0

Train Epoch: 4 [1258/63000 (64%)]	Loss: 0.003837
Train Epoch: 4 [1260/63000 (64%)]	Loss: 0.005010
Train Epoch: 4 [1262/63000 (64%)]	Loss: 0.005726
Train Epoch: 4 [1264/63000 (64%)]	Loss: 0.004966
Train Epoch: 4 [1266/63000 (64%)]	Loss: 0.007417
Train Epoch: 4 [1268/63000 (64%)]	Loss: 0.004191
Train Epoch: 4 [1270/63000 (64%)]	Loss: 0.006861
Train Epoch: 4 [1272/63000 (65%)]	Loss: 0.003025
Train Epoch: 4 [1274/63000 (65%)]	Loss: 0.008333
Train Epoch: 4 [1276/63000 (65%)]	Loss: 0.005594
Train Epoch: 4 [1278/63000 (65%)]	Loss: 0.006816
Train Epoch: 4 [1280/63000 (65%)]	Loss: 0.005600
Train Epoch: 4 [1282/63000 (65%)]	Loss: 0.005343
Train Epoch: 4 [1284/63000 (65%)]	Loss: 0.006390
Train Epoch: 4 [1286/63000 (65%)]	Loss: 0.003233
Train Epoch: 4 [1288/63000 (65%)]	Loss: 0.005548
Train Epoch: 4 [1290/63000 (65%)]	Loss: 0.003511
Train Epoch: 4 [1292/63000 (66%)]	Loss: 0.008841
Train Epoch: 4 [1294/63000 (66%)]	Loss: 0.001659
Train Epoch: 4 [1296/63000 (66%)]	Loss: 0.001584
Train Epoch: 4 [1298

Train Epoch: 4 [1926/63000 (98%)]	Loss: 0.004481
Train Epoch: 4 [1928/63000 (98%)]	Loss: 0.005480
Train Epoch: 4 [1930/63000 (98%)]	Loss: 0.006354
Train Epoch: 4 [1932/63000 (98%)]	Loss: 0.004673
Train Epoch: 4 [1934/63000 (98%)]	Loss: 0.011436
Train Epoch: 4 [1936/63000 (98%)]	Loss: 0.006841
Train Epoch: 4 [1938/63000 (98%)]	Loss: 0.006738
Train Epoch: 4 [1940/63000 (98%)]	Loss: 0.002959
Train Epoch: 4 [1942/63000 (99%)]	Loss: 0.008828
Train Epoch: 4 [1944/63000 (99%)]	Loss: 0.005179
Train Epoch: 4 [1946/63000 (99%)]	Loss: 0.009055
Train Epoch: 4 [1948/63000 (99%)]	Loss: 0.001839
Train Epoch: 4 [1950/63000 (99%)]	Loss: 0.001840
Train Epoch: 4 [1952/63000 (99%)]	Loss: 0.003428
Train Epoch: 4 [1954/63000 (99%)]	Loss: 0.007328
Train Epoch: 4 [1956/63000 (99%)]	Loss: 0.002265
Train Epoch: 4 [1958/63000 (99%)]	Loss: 0.005237
Train Epoch: 4 [1960/63000 (99%)]	Loss: 0.006632
Train Epoch: 4 [1962/63000 (100%)]	Loss: 0.003955
Train Epoch: 4 [1964/63000 (100%)]	Loss: 0.005198
Train Epoch: 4 [19

Train Epoch: 5 [642/63000 (33%)]	Loss: 0.007062
Train Epoch: 5 [644/63000 (33%)]	Loss: 0.006560
Train Epoch: 5 [646/63000 (33%)]	Loss: 0.007986
Train Epoch: 5 [648/63000 (33%)]	Loss: 0.007976
Train Epoch: 5 [650/63000 (33%)]	Loss: 0.007597
Train Epoch: 5 [652/63000 (33%)]	Loss: 0.001627
Train Epoch: 5 [654/63000 (33%)]	Loss: 0.007583
Train Epoch: 5 [656/63000 (33%)]	Loss: 0.006746
Train Epoch: 5 [658/63000 (33%)]	Loss: 0.012398
Train Epoch: 5 [660/63000 (34%)]	Loss: 0.004229
Train Epoch: 5 [662/63000 (34%)]	Loss: 0.003269
Train Epoch: 5 [664/63000 (34%)]	Loss: 0.010165
Train Epoch: 5 [666/63000 (34%)]	Loss: 0.008871
Train Epoch: 5 [668/63000 (34%)]	Loss: 0.008715
Train Epoch: 5 [670/63000 (34%)]	Loss: 0.002241
Train Epoch: 5 [672/63000 (34%)]	Loss: 0.004735
Train Epoch: 5 [674/63000 (34%)]	Loss: 0.004000
Train Epoch: 5 [676/63000 (34%)]	Loss: 0.010908
Train Epoch: 5 [678/63000 (34%)]	Loss: 0.002908
Train Epoch: 5 [680/63000 (35%)]	Loss: 0.004136
Train Epoch: 5 [682/63000 (35%)]	Loss: 0

Train Epoch: 5 [1316/63000 (67%)]	Loss: 0.000653
Train Epoch: 5 [1318/63000 (67%)]	Loss: 0.003311
Train Epoch: 5 [1320/63000 (67%)]	Loss: 0.005032
Train Epoch: 5 [1322/63000 (67%)]	Loss: 0.005230
Train Epoch: 5 [1324/63000 (67%)]	Loss: 0.006491
Train Epoch: 5 [1326/63000 (67%)]	Loss: 0.003784
Train Epoch: 5 [1328/63000 (67%)]	Loss: 0.003818
Train Epoch: 5 [1330/63000 (68%)]	Loss: 0.011071
Train Epoch: 5 [1332/63000 (68%)]	Loss: 0.006427
Train Epoch: 5 [1334/63000 (68%)]	Loss: 0.010019
Train Epoch: 5 [1336/63000 (68%)]	Loss: 0.003197
Train Epoch: 5 [1338/63000 (68%)]	Loss: 0.008125
Train Epoch: 5 [1340/63000 (68%)]	Loss: 0.004856
Train Epoch: 5 [1342/63000 (68%)]	Loss: 0.005367
Train Epoch: 5 [1344/63000 (68%)]	Loss: 0.005706
Train Epoch: 5 [1346/63000 (68%)]	Loss: 0.003149
Train Epoch: 5 [1348/63000 (68%)]	Loss: 0.006280
Train Epoch: 5 [1350/63000 (69%)]	Loss: 0.005495
Train Epoch: 5 [1352/63000 (69%)]	Loss: 0.003302
Train Epoch: 5 [1354/63000 (69%)]	Loss: 0.003696
Train Epoch: 5 [1356

In [19]:
# Partie 4 : Pour aller plus loin

class Conv2DNN(nn.Module):
    def __init__(self, layers_size, activation_function):
        super(Conv2DNN, self).__init__()
        self.activation_function = activation_function
        self.layers = nn.ModuleList()
        #create the layers with a dropout of 0.25 for each convolutionnal layer to prevent overfitting
        for i in range(len(layers_size) - 1):
            layer = nn.Conv2d(layers_size[i], layers_size[i+1], 3, 1)
            self.layers.append(layer)
            layer = nn.Dropout2d(0.25)
            self.layers.append(layer)

    def forward(self, x):
        count = 0
        for l in self.layers:
            if count % 2 == 0:
                x = self.activation_function(l(x))
            elif count == 1:
                x = F.max_pool2d(l(x))
            counter += 1
        return x
            
nb_step = 0.001
epochs = 6
layers_size = [784, 150 ,30, 10]
activation_function = nn.Sigmoid()

conv2dNN = Conv2DNN(layers_size, activation_function)
optimizer = torch.optim.Adam(conv2dNN.parameters(), lr=nb_step)

for epoch in range(epochs):
    conv2dNN.train()
    for batch_ep, (image,label) in enumerate(train_loader):
        optimizer.zero_grad()
        output = conv2dNN(image)
        loss_f = nn.MSELoss()
        loss = loss_f(output, label)
        loss.backward()
        optimizer.step()
        if log:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_ep * len(data), len(train_loader.dataset),
                100. * batch_ep / len(train_loader), loss.item()))
            
conv2dNN.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for image, label in test_loader:
        output = conv2dNN(image)
        lose_f = nn.MSELoss()
        test_loss += lose_f(output, label).item() 
        pred = output.argmax(dim=1, keepdim=True)
        correct += int(np.argmax(output)) == int(np.argmax(label))

test_loss /= len(test_loader.dataset)

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

RuntimeError: Expected 4-dimensional input for 4-dimensional weight 150 784 3 3, but got 2-dimensional input of size [64, 784] instead